In [3]:
import torch
import numpy as np
from botorch.models import SingleTaskGP
from botorch.fit import fit_gpytorch_mll
from botorch.utils import standardize
from gpytorch.mlls import ExactMarginalLogLikelihood

train_X = torch.rand(10, 2, dtype=torch.float64)
Y = 1 - torch.linalg.norm(train_X - 0.5, dim=-1, keepdim=True)
Y = Y + 0.1 * torch.randn_like(Y)  # add some noise
train_Y = standardize(Y)

gp = SingleTaskGP(train_X, train_Y)
mll = ExactMarginalLogLikelihood(gp.likelihood, gp)
fit_gpytorch_mll(mll)
print("-")

-


In [4]:
train_X

tensor([[0.7960, 0.0438],
        [0.9746, 0.4776],
        [0.9788, 0.6750],
        [0.4715, 0.0878],
        [0.6946, 0.3246],
        [0.2092, 0.1813],
        [0.7608, 0.8820],
        [0.9150, 0.6585],
        [0.7536, 0.2738],
        [0.5355, 0.6539]], dtype=torch.float64)

In [5]:
from botorch.acquisition import qExpectedImprovement, qMaxValueEntropy



qEI = qExpectedImprovement(gp,train_Y.max())

In [6]:
from botorch.optim import optimize_acqf

bounds = torch.stack([torch.zeros(2), torch.ones(2)])
candidate, acq_value = optimize_acqf(
    qEI,
    bounds=bounds,
    q=1,
    num_restarts=5,
    raw_samples=20,
)
candidate  # tensor([0.4887, 0.5063])

tensor([[0.4028, 0.6002]])

In [7]:
from typing import Any, List, Optional, Type, Callable, Tuple
from inspect import signature
from botorch.acquisition import AcquisitionFunction
from botorch.acquisition.objective import PosteriorTransform
from botorch.models.gpytorch import Model
from botorch.posteriors import Posterior
from botorch.posteriors.gpytorch import GPyTorchPosterior
from torch import Tensor
from gpytorch.distributions import MultivariateNormal

class SklearnSurrogate(Model):
    # Partially copied from BayBe https://github.com/emdgroup/baybe
    def __init__(self, surrogate) -> None:
        super().__init__()
        self._surrogate = surrogate

    @property
    def num_outputs(self) -> int:
        return 1

    def posterior(
        self,
        X: Tensor,
        output_indices: Optional[List[int]] = None,
        observation_noise: bool = False,
        posterior_transform: Optional[Callable[[Posterior], Posterior]] = None,
        **kwargs: Any
    ) -> Posterior:
        
        x = X.to(torch.float64)
        
        t_shape = x.shape[:-2]
        q_shape = x.shape[-2]
        x_fl = x.flatten(end_dim=-2)

        _mu, _si = self._surrogate.predict(x_fl.numpy(),return_std=True)
        
        mu = torch.from_numpy(_mu)
        si = torch.from_numpy(_si).pow(2)
        
        q_mu = torch.reshape(mu, t_shape + (q_shape,))
        q_si = torch.reshape(si, t_shape + (q_shape,))

        cova = torch.diag_embed(q_si)
        cova.add_(torch.eye(cova.shape[-1]) * 1e-9)
        
        dist = MultivariateNormal(q_mu, cova)
        
        return(GPyTorchPosterior(dist))

In [8]:
from sklearn.linear_model import BayesianRidge
from botorch.optim import optimize_acqf_discrete

brr = BayesianRidge().fit(train_X, train_Y.ravel())

bo_brr = SklearnSurrogate(brr)

qEI = qExpectedImprovement(bo_brr,train_Y.max())

acqf = optimize_acqf_discrete(
    qEI,
    q=2,
    choices=torch.Tensor([[.1,.1],[.2,.2],[.3,.3]]),
)
acqf

(tensor([[0.3000, 0.3000],
         [0.2000, 0.2000]]),
 tensor([0.0077, 0.0145], dtype=torch.float64))

In [9]:
class ForestSurrogate(Model):
    # Partially copied from BayBe https://github.com/emdgroup/baybe
    def __init__(self, surrogate) -> None:
        super().__init__()
        self._surrogate = surrogate

    @property
    def num_outputs(self) -> int:
        return 1

    def posterior(
        self,
        X: Tensor,
        output_indices: Optional[List[int]] = None,
        observation_noise: bool = False,
        posterior_transform: Optional[Callable[[Posterior], Posterior]] = None,
        **kwargs: Any
    ) -> Posterior:
        
        x = X.to(torch.float64)
        
        t_shape = x.shape[:-2]
        q_shape = x.shape[-2]
        x_fl = x.flatten(end_dim=-2)

        preds = np.array([self._surrogate.estimators_[tree].predict(x_fl) for tree in range(self._surrogate.n_estimators)])

        _mu, _si = preds.mean(0), preds.std(0)
        
        mu = torch.from_numpy(_mu)
        si = torch.from_numpy(_si).pow(2)
        
        q_mu = torch.reshape(mu, t_shape + (q_shape,))
        q_si = torch.reshape(si, t_shape + (q_shape,))

        cova = torch.diag_embed(q_si)
        cova.add_(torch.eye(cova.shape[-1]) * 1e-9)
        
        dist = MultivariateNormal(q_mu, cova)
        
        return(GPyTorchPosterior(dist))

In [10]:
from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor().fit(train_X, train_Y.ravel())

bo_rf = ForestSurrogate(rf)

qEI = qExpectedImprovement(bo_rf,train_Y.max())

acqf = optimize_acqf_discrete(
    qEI,
    q=2,
    choices=,
)
acqf

(tensor([[0.3000, 0.3000],
         [0.2000, 0.2000]]),
 tensor([0.0369, 0.0403], dtype=torch.float64))

In [9]:
type(brr)

sklearn.linear_model._bayes.BayesianRidge

In [11]:
cands = torch.Tensor([[.1,.1],[.2,.2],[.3,.3]])

qMES = qMaxValueEntropy(bo_rf, cands)

acqf = optimize_acqf_discrete(
    qMES,
    q=2,
    choices=cands,
)
acqf

AttributeError: 'ForestSurrogate' object has no attribute 'fantasize'